데이터 불러오기

In [3]:
import pandas as pd

# 학습 데이터 불러오기
train_df = pd.read_csv('../data/raw/train.csv')

# 테스트 데이터 불러오기
test_df = pd.read_csv('../data/raw/test.csv')

# 기본 정보 확인
print("train data info:")
train_df.info()

print("\ntest data info:")
test_df.info()


train data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6995056 entries, 0 to 6995055
Data columns (total 3 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   ID      object
 1   URL     object
 2   label   int64 
dtypes: int64(1), object(2)
memory usage: 160.1+ MB

test data info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1747689 entries, 0 to 1747688
Data columns (total 2 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   ID      object
 1   URL     object
dtypes: object(2)
memory usage: 26.7+ MB


결측치 확인

In [4]:
# 각 컬럼별 결측치 개수 확인
train_df.isnull().sum()

# 각 컬럼별 결측치 개수 확인
test_df.isnull().sum()

ID     0
URL    0
dtype: int64

#### URL 문자열 분석

In [6]:
# 1. URL 길이 통계 분석

# train_df와 test_df의 URL 길이 컬럼 생성
train_df['url_length'] = train_df['URL'].str.len()
test_df['url_length'] = test_df['URL'].str.len()

# URL 길이 통계를 확인
print("Train URL 길이 통계:\n", train_df['url_length'].describe())
print("\nTest URL 길이 통계:\n", test_df['url_length'].describe())

Train URL 길이 통계:
 count    6.995056e+06
mean     2.741918e+01
std      3.563130e+01
min      2.000000e+00
25%      1.600000e+01
50%      2.000000e+01
75%      2.900000e+01
max      8.396000e+03
Name: url_length, dtype: float64

Test URL 길이 통계:
 count    1.747689e+06
mean     2.739885e+01
std      3.526333e+01
min      2.000000e+00
25%      1.600000e+01
50%      2.000000e+01
75%      2.900000e+01
max      5.797000e+03
Name: url_length, dtype: float64


In [8]:
# 2. 레이블별 URL 길이 분포 비교

# 악성(label=1)과 정상(label=0) URL의 길이 통계 비교
print("정상 URL 길이 통계:\n", train_df[train_df['label'] == 0]['url_length'].describe())
print("\n정상 URL 길이 통계:\n", train_df[train_df['label'] == 1]['url_length'].describe())

정상 URL 길이 통계:
 count    5.430159e+06
mean     2.096458e+01
std      1.123695e+01
min      2.000000e+00
25%      1.500000e+01
50%      1.900000e+01
75%      2.400000e+01
max      8.320000e+02
Name: url_length, dtype: float64

정상 URL 길이 통계:
 count    1.564897e+06
mean     4.981650e+01
std      6.775459e+01
min      2.000000e+00
25%      2.500000e+01
50%      3.400000e+01
75%      5.000000e+01
max      8.396000e+03
Name: url_length, dtype: float64


다음 단계: URL 구조 통계 분석

In [ ]:
# # 1. URL 파싱(Parsing)

# import urllib.parse as urlparse # URL을 분석하고 조작하는 도구

# # URL 파싱하는 함수 생성
# def parse_url(url):
#     try:
#         # http:// 프로토콜이 없는 경우, 강제로 추가
#         if not urlparse.urlparse(url).scheme:
#             url = 'http://' + url

#         parsed_url = urlparse.urlparse(url)  # URL을 구성 요소별로 분해하는 핵심 함수
#         return {
#             'scheme': parsed_url.scheme,     # http, https 같은 통신 규약 (프로토콜)
#             'netloc': parsed_url.netloc,     # 도메인 (www.example.com)
#             'path': parsed_url.path,         # 파일 또는 경로 (/path/to/file)
#             'params': parsed_url.params,     # 경로 뒤에 붙는 파라미터 (예: /path;params=value)
#             'query': parsed_url.query,       # ? 뒤에 붙는 파라미터 (예: ?id=123&key=value)
#             'fragment': parsed_url.fragment  # # 뒤에 붙는 앵커 (예: #section)
#         }
#     except Exception:
#         return {} # 파싱 실패 시 빈 딕셔너리 반환

# # train_df와 test_df에 파싱 결과를 적용
# train_parsed = train_df['URL'].apply(parse_url)
# test_parsed = test_df['URL'].apply(parse_url)


# # 호스트와 경로 길이를 새로운 컬럼으로 추가
# train_df['host_length'] = train_parsed.apply(lambda x: len(x.get('netloc', '')))
# train_df['path_length'] = train_parsed.apply(lambda x: len(x.get('path', '')))

# # 호스트 길이 통계를 레이블별로 비교
# print("정상 URL 호스트 길이 통계:\n", train_df[train_df['label'] == 0]['host_length'].describe())
# print("\n악성 URL 호스트 길이 통계:\n", train_df[train_df['label'] == 1]['host_length'].describe())

# # 경로 길이 통계를 레이블별로 비교
# print("정상 URL 경로 길이 통계:\n", train_df[train_df['label'] == 0]['path_length'].describe())
# print("\n악성 URL 경로 길이 통계:\n", train_df[train_df['label'] == 1]['path_length'].describe())

In [ ]:
import swifter
from urlextract import URLExtract
import urllib.parse as urlparse
import pandas as pd

# urlextract 객체 생성
extractor = URLExtract()

# URL에서 호스트와 경로를 추출하는 함수
def extract_url_parts(url):
    try:
        urls = extractor.find_urls(url)
        if urls:
            parsed = urlparse.urlparse("http://" + urls[0])
            # 두 값을 튜플로 반환
            return parsed.netloc, parsed.path
        return '', ''
    except TypeError:
        return '', ''

# swifter를 사용하여 데이터프레임에 적용하고 결과를 새로운 컬럼에 할당
# train_df[['host', 'path']] = train_df['URL'].swifter.apply(extract_url_parts, result_type='expand')
# 이 코드를 아래처럼 수정해줘
train_df[['host', 'path']] = train_df['URL'].swifter.apply(extract_url_parts,
                                                            axis=1, # Series의 경우, axis=1를 사용
                                                            # result_type 인자 삭제
                                                           ).apply(pd.Series)


# 이제 'host'와 'path' 컬럼을 기반으로 길이 계산
train_df['host_length'] = train_df['host'].str.len()
train_df['path_length'] = train_df['path'].str.len()

# 호스트 길이 통계를 레이블별로 비교
print("정상 URL 호스트 길이 통계:\n", train_df[train_df['label'] == 0]['host_length'].describe())
print("\n악성 URL 호스트 길이 통계:\n", train_df[train_df['label'] == 1]['host_length'].describe())

# 경로 길이 통계를 레이블별로 비교
print("\n정상 URL 경로 길이 통계:\n", train_df[train_df['label'] == 0]['path_length'].describe())
print("\n악성 URL 경로 길이 통계:\n", train_df[train_df['label'] == 1]['path_length'].describe())

/usr/local/lib/python3.12/site-packages/swifter/swifter.py:307: UserWarning: Axis keyword not necessary because applying on a Series.
  warnings.warn("Axis keyword not necessary because applying on a Series.")


Pandas Apply:   0%|          | 0/6995056 [00:00<?, ?it/s]